<a href="https://colab.research.google.com/github/marciogsantana/Ciencia-de-dados/blob/main/socket_Twitter_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Programa Para o Twitter***

In [ ]:
#cria a seção a ser utiliza para estabelecer a conexão 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkTwitterV02") \
    .getOrCreate()

In [ ]:
#bibliotecas utilizadas para realizar a análise dos textos
from textblob import TextBlob  #utilizada para realizar o processamento do texto e análise de sentimento
from googletrans import Translator #utilizado para traduzir textos 
from unidecode import unidecode  #utilizada para "decodificar caracteres" não textuais

In [ ]:
#cria o dataframe que será responsável por ler cada uma das linhas recebidas através do localhost e porta 9999
# define a fonte (source) de dados
twitters = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9995) \
    .load()

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, FloatType

In [ ]:
#função para realizar a tradução para o inglês 
def translate_udf(col):
    trans_obj=Translator().translate(col)
    return trans_obj.text

#função para realizar a análise de sentimento
def sentiment_udf(col):
    sentiment_text=TextBlob(col)
    return sentiment_text.polarity


In [ ]:
#definição das função como User-Defined-Function
unicode_udf_string = udf(lambda z: unidecode(z), StringType()) #define a função de decode para ser utilizada no dataframe
group_by_sentiment = udf(lambda x: 'negativo' if x < -0.1 else 'positivo' if x > 0.1 else 'neutro',StringType())
translate_udf_string = udf(translate_udf, StringType()) #define a função de tradução
sentiment_udf_float = udf(sentiment_udf, FloatType()) #define a função de tradução


In [ ]:
teste="Eu ♥ o meu cachorro, ele e o meu melhor amigo"
decode=unidecode(teste)
print(decode)
decodeEN=Translator().translate(decode)
print(decodeEN.text)
a=str(decodeEN)
sentiment = TextBlob(a)
print(sentiment.polarity)

Eu  o meu cachorro, ele e o meu melhor amigo
I my dog, he and my best friend
1.0


In [ ]:
#Aplica as funções udf para a seleção de colunas
twitters_unicode=twitters.select( "value",unicode_udf_string(twitters.value).alias("unicoded")) #decodifica
twitters_uni_trans=twitters_unicode.select( "value","unicoded",
                                           translate_udf_string(col("unicoded")).alias("twitter_EN")) #traduz
twitters_uni_trans_sent=twitters_uni_trans.select("value", "unicoded", "twitter_EN", 
                                                 sentiment_udf_float(col("twitter_EN")).alias("analise"))#análise de sentimento
t_sent_label= twitters_uni_trans_sent.select("value","unicoded","twitter_EN", "analise",
                                       group_by_sentiment(col("analise")).alias("classificacao"))

In [ ]:
t_sent_count = t_sent_label.groupBy("classificacao").count() 

In [ ]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado 
query = t_sent_count \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .start()


query.awaitTermination() #aguarda até que a "streaming query" termine 